# pMELTS
  
Versions of MELTS implemented are:  
- MELTS v. 1.0.2 ➞ (rhyolite-MELTS, Gualda et al., 2012)  
- MELTS v. 1.1.0 ➞ (rhyolite-MELTS + new CO<sub>2</sub>, works at the ternary minimum)  
- MELTS v. 1.2.0 ➞ (rhyolite-MELTS + new H<sub>2</sub>O + new CO<sub>2</sub>)  
- pMELTS v. 5.6.1

## Initialize tools and packages that are required to execute this notebook.

In [ ]:
from thermoengine import equilibrate
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Create a pMELTS v 5.6.1 instance.
Rhyolite-MELTS version 1.0.2 is the default model.

In [ ]:
melts = equilibrate.MELTSmodel(version="5.6.1")

### Optional: Generate some information about the implemented model.

In [ ]:
oxides = melts.get_oxide_names()
phases = melts.get_phase_names()
#print (oxides)
#print (phases)

## Required: Input initial composition of the system (liquid), in wt% or grams of oxides. 
Early Bishop Tuff average melt inlusion composition

In [ ]:
feasible = melts.set_bulk_composition({'SiO2':  45.47, 
                                       'TiO2':   0.11, 
                                       'Al2O3':  4.0, 
                                       'Fe2O3':  0.585,
                                       'Cr2O3':  0.0, 
                                       'FeO':    6.696, 
                                       'MnO':    0.0,
                                       'MgO':   38.53, 
                                       'NiO':    0.0, 
                                       'CoO':    0.0,
                                       'CaO':    3.59, 
                                       'Na2O':   0.31, 
                                       'K2O':    0.0, 
                                       'P2O5':   0.0, 
                                       'H2O':    0.0})

## Optional: Suppress phases that are not required in the simulation.

In [ ]:
b = melts.get_phase_inclusion_status()
melts.set_phase_inclusion_status({'Actinolite':False, 'Aegirine':False, \
                                  'Aenigmatite':False, 'Akermanite':False, 'Andalusite':False, \
                                  'Anthophyllite':False, 'Apatite':False, 'Biotite':False, 'Chromite':False, \
                                  'Coesite':False, 'Corundum':False, 'Cristobalite':False, 'Cummingtonite':False, \
                                  'Fayalite':False, 'Forsterite':False, 'Gehlenite':False, 'Hematite':False, \
                                  'Hornblende':False, 'Ilmenite':False, 'Ilmenite ss':False, 'Kalsilite':False, \
                                  'Kalsilite ss':False, 'Kyanite':False, 'Leucite':False, 'Lime':False, \
                                  'Liquid Alloy':False, 'Magnetite':False, 'Melilite':False, 'Muscovite':False, \
                                  'Nepheline':False, 'Nepheline ss':False, 'OrthoOxide':False, 'Panunzite':False, \
                                  'Periclase':False, 'Perovskite':False, 'Phlogopite':False, 'Quartz':False, \
                                  'Rutile':False, 'Sanidine':False, 'Sillimanite':False, 'Solid Alloy':False, \
                                  'Sphene':False, 'Tridymite':False, 'Whitlockite':False})

a = melts.get_phase_inclusion_status()
for phase in b.keys():
    if b[phase] != a[phase]:
        print ("{0:<15s} Before: {1:<5s} After: {2:<5s}".format(phase, repr(b[phase]), repr(a[phase])))

## Compute the equilibrium state at some specified T (°C) and P (MPa).
Print status of the calculation.

In [ ]:
output = melts.equilibrate_tp(1300.0, 1000.0, initialize=True)
(status, t, p, xmlout) = output[0]
print (status, t, p)

### Summary output of equilibrium state ...

In [ ]:
melts.output_summary(xmlout)

### Output thermodynamic properties of any phase present in the system
... or the sum of all phases in the system

In [ ]:
print ("{0:<20s} {1:13.6e} {2:<10s}".format('Entropy', melts.get_property_of_phase(xmlout,'System', 'Entropy'), \
                                            melts.get_units_of_property('Entropy')))

## Run the sequence of calculations along a T, P gradient:
Output is sent to an Excel file and plotted in the notebook

In [ ]:
number_of_steps = 20
t_increment_of_steps = -5.0
p_increment_of_steps = -50.0

plotPhases = ['Liquid', 'Olivine', 'Orthopyroxene', 'Augite', 'Plagioclase']
# matplotlib colors b : blue, g : green, r : red, c : cyan, m : magenta, y : yellow, k : black, w : white.
plotColors = [ 'ro', 'bo', 'go', 'co', 'mo']

wb = melts.start_excel_workbook_with_sheet_name(sheetName="Summary")
melts.update_excel_workbook(wb, xmlout)

n = len(plotPhases)
xPlot = np.zeros(number_of_steps)
yPlot = np.zeros((n, number_of_steps))
y2Plot = np.full(number_of_steps, t)
xPlot[0] = p
for i in range (0, n):
    yPlot[i][0] = melts.get_property_of_phase(xmlout, plotPhases[i])
y2Plot[0] = t

plt.ion()
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim([min(p, p+p_increment_of_steps*number_of_steps), max(p, p+p_increment_of_steps*number_of_steps)])
ax.set_ylim([0., 100.])
ax2 = ax.twinx()
ax2.set_ylabel('T', color='k')
ax2.set_ylim([t-100, t])

graphs = []
for i in range (0, n):
    graphs.append(ax.plot(xPlot, yPlot[i], plotColors[i]))
graphs.append(ax2.plot(xPlot, y2Plot, 'k-'))
handle = []
for (graph,) in graphs:
    handle.append(graph)
ax.legend(handle, plotPhases, loc='upper left')

for i in range (1, number_of_steps):
    output = melts.equilibrate_tp(t+t_increment_of_steps, p+p_increment_of_steps)
    (status, t, p, xmlout) = output[0]
    print ("{0:<30s} {1:8.2f} {2:8.2f}".format(status, t, p))
    xPlot[i] = p
    for j in range (0, n):
        yPlot[j][i] = melts.get_property_of_phase(xmlout, plotPhases[j])
    y2Plot[i] = t
    j = 0
    for (graph,) in graphs:
        graph.set_xdata(xPlot)
        if j < n:
            graph.set_ydata(yPlot[j])
        else:
            graph.set_ydata(y2Plot)
        j = j + 1
    fig.canvas.draw()
    melts.update_excel_workbook(wb, xmlout)

melts.write_excel_workbook(wb, "MELTSv102summary.xlsx")